## 1. 209 - Minimum Size Subarray Sum
Given an array of n positive integers and a positive integer s, find the minimal length of a contiguous subarray of which the sum ≥ s. If there isn't one, return 0 instead.

**idea** two pointers, one pass  
if sum < s, move fast pointer to right  
if sum >= s, move slow pointer to right  
remember to memorize the total value

In [1]:
class Solution(object):
    def minSubArrayLen(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        if not nums: return 0
        i, j, total, minlen = 0, 1, 0, len(nums)+1
        while i < j <= len(nums):
            total += nums[j-1]
            while total >= s:
                minlen = min(j-i, minlen)
                total -= nums[i]
                i += 1
            j += 1
        return 0 if minlen == len(nums)+1 else minlen

**idea** dfs  
for every num in the array, take it as the right end  
find the left end of it

In [2]:
class Solution(object):
    def minSubArrayLen(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        if not nums: return 0
        for i,x in enumerate(nums[1:],1):
            nums[i] = nums[i-1] + x
        minlen = len(nums)+1
        for r,x in enumerate(nums):
            if x >= s:
                l = self.bs(nums,s,r,x)
                minlen = min(r-l+1, minlen)
        return minlen if minlen <= len(nums) else 0
        
    def bs(self,nums,s,r,x):
        l = 0
        while l<r:
            mid = (l+r)/2
            if x-nums[mid] >= s:
                l = mid + 1
            else:
                r = mid
        return l           

## 2. 211 - Add and Search Word - Data structure design
Design a data structure that supports the following two operations:  
void addWord(word)  
bool search(word)  
search(word) can search a literal word or a regular expression string containing only letters a-z or .. A . means it can represent any one letter.

**idea** dictionary  
key: length of word, value: word  
search every word with corresponding length

In [3]:
class WordDictionary(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.dic = collections.defaultdict(list)

    def addWord(self, word):
        """
        Adds a word into the data structure.
        :type word: str
        :rtype: void
        """
        self.dic[len(word)].append(word)

    def search(self, word):
        """
        Returns if the word is in the data structure. A word could contain the dot character '.' to represent any one letter.
        :type word: str
        :rtype: bool
        """
        for v in self.dic[len(word)]:
            for i,ch in enumerate(word):
                if ch != '.' and ch != v[i]:
                    break
                # we see a wrong word, break this loop, search another word
            else: return True
        return False


# Your WordDictionary object will be instantiated and called as such:
# obj = WordDictionary()
# obj.addWord(word)
# param_2 = obj.search(word)

**idea** trie

In [4]:
class TrieNode(object):
    
    def __init__(self):
        self.isword = False
        self.children = {}
    
class WordDictionary(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = TrieNode()

    def addWord(self, word):
        """
        Adds a word into the data structure.
        :type word: str
        :rtype: void
        """
        node = self.root
        for ch in word:
            if ch not in node.children:
                node.children[ch] = TrieNode()
            node = node.children[ch]
        node.isword = True
            

    def search(self, word):
        """
        Returns if the word is in the data structure. A word could contain the dot character '.' to represent any one letter.
        :type word: str
        :rtype: bool
        """
        return self.helper(self.root, word)
    
    def helper(self, node, word):
        for i,ch in enumerate(word):
            if ch != '.' and ch not in node.children:
                return False
            elif ch in node.children:
                node = node.children[ch]
            elif ch == '.':
                for child in node.children:
                    if self.helper(node.children[child], word[i+1:]):
                        return True
                return False
        return node.isword
                


# Your WordDictionary object will be instantiated and called as such:
# obj = WordDictionary()
# obj.addWord(word)
# param_2 = obj.search(word)

## 3. 213 - House Robber II
You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed. All houses at this place are **arranged in a circle**. That means the first house is the neighbor of the last one. Meanwhile, adjacent houses have security system connected and it will automatically contact the police if two adjacent houses were broken into on the same night.  
Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight without alerting the police.

**idea** dp, but divide into two cases:  
1. rob the first house
2. don't rob the first house

In [5]:
class Solution(object):
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) == 1: return nums[0]
        # divide into two cases: house 0 is robbed or not
        return max(self.helper(nums[1:]), self.helper(nums[:-1]))
        
    def helper(self, nums):
        pre, cur = 0, 0
        for num in nums:
            pre, cur = cur, max(pre+num, cur)
        return cur

## 4. 337 - House Robber III
The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". It will automatically contact the police **if two directly-linked houses** were broken into on the same night. Determine the maximum amount of money the thief can rob tonight without alerting the police.

**idea** dfs. Define a subrob function, first param is optimal if we can rob root, second param is optimal if we don't rob root at all  
0. if we rob root, and don't rob left and right node
1. if we don'r rob root, and can rob left and right node

In [6]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def rob(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        return self.subrob(root)[0]
    
    
    def subrob(self, root):
        # 0: with root
        # 1: without root
        if not root:
            return (0,0)
        left = self.subrob(root.left)
        right = self.subrob(root.right)
        return (max(left[1]+right[1]+root.val, left[0]+right[0]), left[0]+right[0])

### 5. 414 - Third Maximum Number
**idea** memorize three largest number, compare every new number to the third largest one

In [7]:
class Solution(object):
    def thirdMax(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        ret, mini = [], 0
        for num in nums:
            if num not in ret:
                if len(ret) < 3:
                    ret.append(num)
                    mini = min(ret)
                else:
                    if num > mini:
                        ret.remove(mini)
                        ret.append(num)
                        mini = min(ret)
        return mini if len(ret) == 3 else max(nums)